In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17710403
paper_name = 'defilippi_deloche_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/Table2.txt', sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 43 x 3


In [7]:
original_data.head()

,Unnamed: 0,10 uM,20 uM
0,SAC1,−,−
1,VPS1,−,−
2,VPS54,−,−
3,VPS52,−,−
4,PEP5,−,−


In [8]:
original_data['gene'] = original_data['Unnamed: 0'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Unnamed: 0 10 uM 20 uM  gene   orf
index_input                                   
23                 PTH   +/−   +/−   PTH   PTH
34                STA1   +/−     −  STA1  STA1


In [13]:
original_data = original_data.loc[t,:]

In [16]:
original_data.columns

Index(['Unnamed: 0', '10 uM', '20 uM', 'gene', 'orf'], dtype='object')

In [20]:
data_switch = {'−': -2, '+/−': -1, '+': 0}
for c in ['10 uM', '20 uM']:
    original_data[c] = original_data[c].apply(lambda x: data_switch[x.replace(u'\xa0', '')])

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
original_data.set_index('orf', inplace=True)

In [23]:
original_data = original_data[['10 uM', '20 uM']].copy()

In [24]:
original_data = original_data.groupby(original_data.index).mean()

In [25]:
original_data.shape

(40, 2)

# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [16688, 16689]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,16688,16689
data_type,value,value
orf,,
YAL026C,-1,-2
YAL035W,-1,-1
YBL058W,-1,-2
YBR111C,-1,-1
YBR201W,-1,-2


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16688,16689
,data_type,value,value
gene_id,orf,,
24,YAL026C,-1,-2
33,YAL035W,-1,-1
146,YBL058W,-1,-2
305,YBR111C,-1,-1
395,YBR201W,-1,-2


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16688,16689,16688,16689
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
24,YAL026C,-1,-2,-10.173495,-13.29866
33,YAL035W,-1,-1,-10.173495,-6.64933
146,YBL058W,-1,-2,-10.173495,-13.29866
305,YBR111C,-1,-1,-10.173495,-6.64933
395,YBR201W,-1,-2,-10.173495,-13.29866


# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 17710403...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  4.45it/s]

Updating the data_modified_on field...
